In [1]:
!pip install torch
import torch
import torch.nn as nn
from torch.nn import functional as F 
import pandas as pd 
import numpy as np 

In [2]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')


In [3]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


In [4]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [9]:
from sklearn.model_selection import train_test_split
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, -1].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, random_state = 123)

In [10]:
# scale and transform 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train_ = torch.from_numpy(X_train).float()
X_test_ = torch.from_numpy(X_test).float()

In [11]:
y_train = torch.from_numpy(Y_train)
y_test = torch.from_numpy(Y_test)

In [12]:
y_train

tensor([1, 0, 1,  ..., 1, 1, 1])

In [15]:
input_size =2
output_size = 2
hidden_size = 10

In [16]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size, output_size)


    def forward(self, X):
        X = torch.relu((self.fc1(X)))
        X = torch.relu((self.fc2(X)))
        X = self.fc3(X)

        return F.log_softmax(X,dim=1)

model = Net()

In [17]:
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

epochs = 100

In [19]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(X_train_)
  loss = loss_fn(Ypred,  y_train)
  loss.backward()
  optimizer.step()
  print('Epoch',epoch, 'loss',loss.item())

list(model.parameters())

Epoch 0 loss 0.6584901809692383
Epoch 1 loss 0.6353955864906311
Epoch 2 loss 0.6140943765640259
Epoch 3 loss 0.5936009883880615
Epoch 4 loss 0.5735892057418823
Epoch 5 loss 0.5541473627090454
Epoch 6 loss 0.5350092053413391
Epoch 7 loss 0.5158778429031372
Epoch 8 loss 0.4962187707424164
Epoch 9 loss 0.4757603406906128
Epoch 10 loss 0.45457878708839417
Epoch 11 loss 0.4328427314758301
Epoch 12 loss 0.41081175208091736
Epoch 13 loss 0.38915663957595825
Epoch 14 loss 0.3681758940219879
Epoch 15 loss 0.34760379791259766
Epoch 16 loss 0.3275575041770935
Epoch 17 loss 0.3082078993320465
Epoch 18 loss 0.2896336317062378
Epoch 19 loss 0.2719314694404602
Epoch 20 loss 0.25525492429733276
Epoch 21 loss 0.23967255651950836
Epoch 22 loss 0.22522716224193573
Epoch 23 loss 0.2119283229112625
Epoch 24 loss 0.19966739416122437
Epoch 25 loss 0.18855108320713043
Epoch 26 loss 0.17868904769420624
Epoch 27 loss 0.1698390394449234
Epoch 28 loss 0.1619032621383667
Epoch 29 loss 0.15495015680789948
Epoch 30 

[Parameter containing:
 tensor([[ 0.2675,  0.6899],
         [-0.8905,  0.7650],
         [ 0.9741, -0.7869],
         [-0.8390, -0.9023],
         [-0.8747, -0.9065],
         [-0.6077, -0.8020],
         [ 0.9068,  0.9137],
         [-0.3334, -0.4592],
         [ 0.6652,  0.5548],
         [-0.6564, -0.0278]], requires_grad=True),
 Parameter containing:
 tensor([ 0.2960,  0.4198, -0.3629,  0.6986,  0.0825,  0.1627, -0.3019,  1.0874,
          0.4572, -0.1215], requires_grad=True),
 Parameter containing:
 tensor([[ 3.5906e-01,  2.7795e-01,  5.2886e-01,  2.4881e-01, -5.3731e-01,
          -6.8811e-01,  6.7005e-01,  2.1489e-01,  1.6044e-01,  5.7774e-03],
         [ 2.2177e-01,  4.9093e-01,  8.0664e-01, -1.8004e-01, -5.2820e-01,
          -3.9550e-01,  1.5304e-01,  5.2354e-01,  4.7654e-01,  1.7096e-01],
         [-2.4963e-01, -2.9390e-01,  7.2783e-04, -2.8266e-01, -9.4181e-03,
          -8.0866e-02, -2.4126e-01, -5.0786e-02, -7.9373e-02,  2.4368e-01],
         [-4.9182e-01, -1.0697e-01, 

In [20]:
torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()

y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0056, -5.1939]], grad_fn=<LogSoftmaxBackward>)

In [21]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

y_cust_42_50000 = model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

torch.save(model,'customer_buy.pt')

In [22]:
restored_model = torch.load('customer_buy.pt')
